In [1]:
#setup - rememeber to switch to tensorflow 2.3 kernel...
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io as sio
import datetime
import trimesh

#need to have these two lines to work on my ancient 1060 3gb
#  https://stackoverflow.com/questions/43990046/tensorflow-blas-gemm-launch-failed
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

print(tf.__version__)

# %matplotlib inline
# plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
# plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams['image.cmap'] = 'gray'
%matplotlib notebook

%load_ext tensorboard

# for auto-reloading external modules
%load_ext autoreload
%autoreload 2
%autosave 180

C:\Users\Derm\anaconda3\envs\tf23\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Derm\anaconda3\envs\tf23\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Derm\anaconda3\envs\tf23\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


2.3.0


Autosaving every 180 seconds


In [32]:
#load OFF file from ModelNet10 dir
fn = 'C:/Users/Derm/Desktop/big/ModelNet10/toilet/train/toilet_0069.off'
M = trimesh.load(fn)
print(M.vertices)

test = trimesh.sample.sample_surface(M, 100)
# print(test[0])

[[-8.255375  8.5247   11.527942]
 [-8.005575  8.97041  11.869442]
 [-8.395175  8.87073  11.872142]
 ...
 [ 0.581825 -8.34001   2.091742]
 [-2.302875 -9.21081   2.009842]
 [-2.303875 -5.95071   2.110942]]


In [33]:
#use Vedo to plot OG and subsampled surfaces
from vedo import *
from ipyvtklink.viewer import ViewInteractiveWidget

plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp = []

disp.append(Points(M.vertices, c = 'blue', r = 4))
disp.append(Points(test[0], c = 'red', r = 10))

plt.show(disp, "surface sampling test")
ViewInteractiveWidget(plt.window)

ViewInteractiveWidget(height=960, layout=Layout(height='auto', width='100%'), width=960)